# empty solar

In [11]:
import requests

# Replace with your actual Solr core name
solr_core = "mycore"
solr_url = f"http://localhost:8983/solr/{solr_core}/update"

# Delete all documents
delete_all_query = {
    "delete": {"query": "*:*"}
}

# Send delete request
response = requests.post(solr_url, json=delete_all_query, headers={"Content-type": "application/json"})

if response.status_code == 200:
    print("All documents deleted successfully.")
else:
    print(f"Failed to delete documents: {response.text}")

# Commit the changes
commit_url = f"http://localhost:8983/solr/{solr_core}/update?commit=true"
commit_response = requests.get(commit_url)

if commit_response.status_code == 200:
    print("Changes committed.")
else:
    print(f"Failed to commit changes: {commit_response.text}")


All documents deleted successfully.
Changes committed.


In [5]:
import requests
import json

# Solr core name
core_name = "mycore"

# Solr update endpoint (note `/json/docs` here)
url = f"http://localhost:8983/solr/{core_name}/update/json/docs?commit=true"

# Load your unmodified single JSON object
with open("/home/pa2/ml/tf_gpu/OCR/D:/BPL/BLS/Result/BPL- TM Applications.json", "r") as f:
    doc = json.load(f)

# Add an ID if needed
if 'id' not in doc:
    doc['id'] = 'doc1'  # Solr requires an 'id' field

# Send it to Solr
headers = {"Content-Type": "application/json"}
response = requests.post(url, headers=headers, json=doc)

# Response check
if response.status_code == 200:
    print("Document uploaded successfully.")
else:
    print(f"Failed to upload document. Status: {response.status_code}")
    print(response.text)


Document uploaded successfully.


In [24]:
import requests
import json

# Solr configuration
core_name = "mycore"  # Replace with actual core name
solr_url = f"http://localhost:8983/solr/{core_name}/select"
params = {
    "q": "*:*",
    "wt": "json",
    "rows": 2
}

# Query Solr
response = requests.get(solr_url, params=params)
data = response.json()
doc = data["response"]["docs"][0]

# Fix section_embedding
# flat_embedding = doc.get("section_embedding", [])
# embedding_dim = 1024

# # Reconstruct 2D array
# if len(flat_embedding) % embedding_dim != 0:
#     raise ValueError("Flat embedding length is not divisible by embedding dimension.")

# num_vectors = len(flat_embedding) // embedding_dim
# section_embedding_2d = [
#     flat_embedding[i * embedding_dim: (i + 1) * embedding_dim]
#     for i in range(num_vectors)
# ]

# Construct result
# result = {
#     "class": doc.get("class", [""])[0],
#     "text": doc.get("text", [""])[0],
#     "section_embedding": section_embedding_2d,
#     "total_embedding": doc.get("total_embedding", [])
# }

# Save to JSON
# output_json = "solr_output.json"
# with open(output_json, "w", encoding="utf-8") as f:
#     json.dump(result, f, indent=2)

# print(f"Saved structured result to: {output_json}")


In [27]:
len(doc['total_embedding'])

1024

In [18]:
import os
import json
import requests
SOLR_URL = "http://localhost:8983/solr/mycore/update/json/docs"
HEADERS = {"Content-Type": "application/json"}
JSON_DIR = "/home/pa2/ml/tf_gpu/OCR/D:/BPL/BLS/Result/"
def upload_json_files_to_solr(json_dir):
    for filename in os.listdir(json_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(json_dir, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                doc = json.load(f)
            if "id" not in doc:
                print(f"[SKIP] Missing 'id' in: {filename}")
                continue
            if "section_embedding" in doc and isinstance(doc["section_embedding"], list):
                if doc["section_embedding"] and isinstance(doc["section_embedding"][0], (int, float)):
                    doc["section_embedding"] = [doc["section_embedding"]]
            response = requests.post(SOLR_URL + "?commit=true", headers=HEADERS, json=doc)
            if response.status_code == 200:
                print(f"[OK] Uploaded: {filename}")
            else:
                print(f"[FAIL] {filename} - {response.status_code}: {response.text}")
upload_json_files_to_solr(JSON_DIR)

[OK] Uploaded: BPL- TM Applications.json
[OK] Uploaded: Alvira_Reality_Services_Pvt_Ltd_vs_Govt_Of_Nct_Of_Delhi_Anr_on_28_March_2025.json


In [2]:
import requests
import json
import os
core_name = "mycore"
solr_url = f"http://localhost:8983/solr/{core_name}/select"
initial_params = {
    "q": "*:*",
    "wt": "json",
    "rows": 0
}
response = requests.get(solr_url, params=initial_params)
total_docs = response.json()["response"]["numFound"]
params = {
    "q": "*:*",
    "wt": "json",
    "rows": total_docs
}
response = requests.get(solr_url, params=params)
docs = response.json()["response"]["docs"]

In [64]:
def embedding_conversion(li,embedding_dim=1024):
    flat_embedding=li
    if len(flat_embedding) % embedding_dim != 0:
        raise ValueError("Flat embedding length is not divisible by embedding dimension.")
    
    num_vectors = len(flat_embedding) // embedding_dim
    section_embedding_2d = [
        flat_embedding[i * embedding_dim: (i + 1) * embedding_dim]
        for i in range(num_vectors)
    ]
    return section_embedding_2d

In [67]:
dic={}
docs1=docs
for i in range(len(docs)):
    # print(docs[i]["section_embedding"])
    li=[docs[i]['class'][0],docs[i]['text'][0],embedding_conversion(docs[i]["section_embedding"]),docs[i]['total_embedding']]
    dic[i]=li
    # break

In [68]:
docs1[0].keys()

dict_keys(['id', 'class', 'text', 'section_embedding', 'total_embedding', '_version_'])

In [69]:
docs1[0]['id']

'BPL- TM Applications'

In [83]:
import fitz
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from doctr.models import ocr_predictor
from doctr.io import DocumentFile
from PIL import Image
import torch
from transformers import VisionEncoderDecoderModel, TrOCRProcessor,BitsAndBytesConfig
from pdf2image import convert_from_path
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.preprocessing import StandardScaler
import torch
import json
import csv
import threading
import requests

In [84]:
paths=[
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/1.Contractual Document.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/2.Regulatory and Compliance Sub Categories.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/3.IPR Categories.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/4.Litigation and Dispute Categories.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/5.Property and Real Estate.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/6.Corporate and Governance Categories.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/7.HR Document categories.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/8.Finance and Banking Categories.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/9.Government and public laws.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/10.Personal and Civil Legal Documents.txt",
    "/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/11.Bill of Entry.txt"
]
emb_dict={}
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0
    )
model=HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
)
for i in paths:
    name=i.split('/')[-1].split('.')[1]
    print("Processing",name,"Document")
    with open(i,'r') as file:
        new_context=file.read()
    docs=text_splitter.split_text(new_context)
    docs=model.embed_documents(docs)
    doc=np.zeros(len(docs[0]))
    for j in docs:
        doc+=j
    emb_dict[i]=doc


Processing Contractual Document Document
Processing Regulatory and Compliance Sub Categories Document
Processing IPR Categories Document
Processing Litigation and Dispute Categories Document
Processing Property and Real Estate Document
Processing Corporate and Governance Categories Document
Processing HR Document categories Document
Processing Finance and Banking Categories Document
Processing Government and public laws Document
Processing Personal and Civil Legal Documents Document
Processing Bill of Entry Document


In [85]:
emb_dict

{'/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/1.Contractual Document.txt': array([ 3.44799794, -7.76196136,  6.94890799, ..., -6.92812096,
         7.62558395,  5.5105202 ], shape=(1024,)),
 '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/2.Regulatory and Compliance Sub Categories.txt': array([  2.22487216, -12.26199993,   8.47237386, ...,  -8.74275181,
          8.17517548,   2.986623  ], shape=(1024,)),
 '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/3.IPR Categories.txt': array([  1.27148668, -10.18916291,  10.42277481, ...,  -7.07592817,
          9.81600397,   5.54344384], shape=(1024,)),
 '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/4.Litigation and Dispute Categories.txt': array([  2.09481126, -12.37001365,  11.96785082, ..., -10.0581905 ,
         10.8287801 ,  -1.10811053], shape=(1024,)),
 '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/5.Property and Real Estate.txt': array([ 1.50327174, -5.66086469,  7.81525035, 

In [86]:
import requests

# Configuration
SOLR_URL = "http://localhost:8983/solr/admin/cores"
CORE_NAME = "emb_core"  # Name of the core you want to create
INSTANCE_DIR = CORE_NAME   # Use same name for instanceDir by convention

params = {
    "action": "CREATE",
    "name": CORE_NAME,
    "instanceDir": INSTANCE_DIR,
    "configSet": "_default",  # Or use a custom config set if you've uploaded one
    "numShards": 1
}

# Send request
response = requests.get(SOLR_URL, params=params)

# Result
if response.status_code == 200 and "responseHeader" in response.json():
    print(f"[OK] Core '{CORE_NAME}' created successfully.")
else:
    print(f"[FAIL] Core creation failed: {response.status_code}")
    print(response.text)


[FAIL] Core creation failed: 400
{
  "responseHeader":{
    "status":400,
    "QTime":7},
  "error":{
    "metadata":[
      "error-class","org.apache.solr.common.SolrException",
      "root-error-class","org.apache.solr.common.SolrException"],
    "msg":"Error CREATEing SolrCore 'emb_core': Unable to create core [emb_core] Caused by: Could not load configuration from directory /var/solr/data/configsets/_default",
    "code":400}}



In [87]:
import requests

core_name = "emb_core"
solr_url = f"http://localhost:8983/solr/admin/cores?action=UNLOAD&core={core_name}&deleteIndex=true&deleteDataDir=true&deleteInstanceDir=true"

response = requests.get(solr_url)

if response.status_code == 200:
    print(f"[OK] Core '{core_name}' deleted successfully.")
else:
    print(f"[FAIL] Failed to delete core '{core_name}': {response.status_code}")
    print(response.text)


[OK] Core 'emb_core' deleted successfully.


In [108]:
import requests
import numpy as np

# ——— Your emb_dict example ———
# emb_dict: { filepath (str) → np.ndarray(shape=(1024,)) }
# (You already have this in your notebook.)

# ——— Solr configuration ———
CORE = "emb_core"
SOLR_URL = f"http://localhost:8983/solr/{CORE}/update?commit=true"
HEADERS  = {"Content-Type": "application/json"}

def index_emb_dict(emb_dict):
    # Build a batch of Solr docs
    docs = []
    for path, arr in emb_dict.items():
        vec = arr.tolist()  # convert numpy → plain Python list
        docs.append({
            "id": path,                 # unique ID
            "filepath": path,
            "total_embedding": vec,     # 1D embedding
            # if you actually want a 2D “section_embedding”:
            # "section_embedding": [vec],
        })

    # Send all docs in one bulk update
    print(len(docs))
    resp = requests.post(SOLR_URL, headers=HEADERS, json=docs)
    if resp.ok:
        print(f"[OK] Indexed {len(docs)} documents.")
    else:
        print(f"[FAIL] {resp.status_code}\n{resp.text}")

# ——— Call it! ———
index_emb_dict(emb_dict)


11
[OK] Indexed 11 documents.


In [110]:
import requests
import numpy as np

# ——— Solr configuration ———
CORE     = "emb_core"  # or "mycore", whatever your core is called
BASE_URL = f"http://localhost:8983/solr/{CORE}/select"

# 1) Fetch the total number of docs
resp0 = requests.get(BASE_URL, params={"q": "*:*", "wt": "json", "rows": 0})
resp0.raise_for_status()
total = resp0.json()["response"]["numFound"]

# 2) Fetch all docs in one go
resp1 = requests.get(
    BASE_URL,
    params={"q": "*:*", "wt": "json", "rows": total}
)
resp1.raise_for_status()
docs = resp1.json()["response"]["docs"]

# 3) Reconstruct emb_dict
emb_dict1 = {}
for d in docs:
    path = d["id"]
    # pull the stored list and turn it back into an ndarray
    vec  = d.get("total_embedding", [])
    emb_dict1[path] = np.array(vec, dtype=float)

# ✔️ Now emb_dict[path] is your original np.array(shape=(1024,))
print(f"Reconstructed emb_dict with {len(emb_dict1)} entries.")


Reconstructed emb_dict with 11 entries.


In [119]:
for i in emb_dict1:
    print(emb_dict[i])

[ 3.44799794 -7.76196136  6.94890799 ... -6.92812096  7.62558395
  5.5105202 ]
[  2.22487216 -12.26199993   8.47237386 ...  -8.74275181   8.17517548
   2.986623  ]
[  1.27148668 -10.18916291  10.42277481 ...  -7.07592817   9.81600397
   5.54344384]
[  2.09481126 -12.37001365  11.96785082 ... -10.0581905   10.8287801
  -1.10811053]
[ 1.50327174 -5.66086469  7.81525035 ... -8.4682214   8.55662783
  0.85942307]
[  3.63572187 -11.97567185   7.37724581 ...  -8.59058407   8.29276994
   3.06223828]
[  2.38134082 -14.4132777    8.78386503 ...  -9.05948943   8.13576377
   5.5455265 ]
[ 3.9478638  -9.30388876  9.08887558 ... -8.91509168 10.11874479
  1.91462602]
[  2.78824287 -10.31885303   9.93410228 ...  -9.12498925   6.80312538
   1.11983385]
[  1.79554164 -11.32311716   9.06838632 ...  -8.61582864  10.52319273
   1.54119968]
[ 0.4623978  -2.60747719  1.96213558 ... -2.0259838   1.77185509
  0.63901934]


In [121]:
print(emb_dict1)

{'/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/1.Contractual Document.txt': array([ 3.44799794, -7.76196136,  6.94890799, ..., -6.92812096,
        7.62558395,  5.5105202 ], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/2.Regulatory and Compliance Sub Categories.txt': array([  2.22487216, -12.26199993,   8.47237386, ...,  -8.74275181,
         8.17517548,   2.986623  ], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/3.IPR Categories.txt': array([  1.27148668, -10.18916291,  10.42277481, ...,  -7.07592817,
         9.81600397,   5.54344384], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/4.Litigation and Dispute Categories.txt': array([  2.09481126, -12.37001365,  11.96785082, ..., -10.0581905 ,
        10.8287801 ,  -1.10811053], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/5.Property and Real Estate.txt': array([ 1.50327174, -5.66086469,  7.81525035, ..., -8.

In [122]:
print(emb_dict)

{'/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/1.Contractual Document.txt': array([ 3.44799794, -7.76196136,  6.94890799, ..., -6.92812096,
        7.62558395,  5.5105202 ], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/2.Regulatory and Compliance Sub Categories.txt': array([  2.22487216, -12.26199993,   8.47237386, ...,  -8.74275181,
         8.17517548,   2.986623  ], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/3.IPR Categories.txt': array([  1.27148668, -10.18916291,  10.42277481, ...,  -7.07592817,
         9.81600397,   5.54344384], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/4.Litigation and Dispute Categories.txt': array([  2.09481126, -12.37001365,  11.96785082, ..., -10.0581905 ,
        10.8287801 ,  -1.10811053], shape=(1024,)), '/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/5.Property and Real Estate.txt': array([ 1.50327174, -5.66086469,  7.81525035, ..., -8.